# Fine-tuning a model with the Trainer API or Keras

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [12]:
# Instalasi library yang diperlukan
!pip install datasets evaluate transformers[sentencepiece]

In [13]:
# Memuat dataset GLUE (MRPC)
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")  # Dataset GLUE dengan task MRPC (paraphrase detection)

# Checkpoint model yang digunakan
checkpoint = "bert-base-uncased"
# Memuat tokenizer berdasarkan checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Fungsi tokenisasi untuk dataset
def tokenize_function(example):
    # Tokenisasi dua kalimat dengan truncation
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Memetakan fungsi tokenisasi ke seluruh dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Menggunakan DataCollator untuk padding dinamis
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [14]:
# Mengatur parameter untuk pelatihan
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")  # Direktori output untuk hasil pelatihan

In [15]:
# Memuat model untuk klasifikasi urutan dengan dua label (0 dan 1)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Menggunakan Trainer API untuk pelatihan
from transformers import Trainer

trainer = Trainer(
    model=model,  # Model yang digunakan
    args=training_args,  # Parameter pelatihan
    train_dataset=tokenized_datasets["train"],  # Dataset untuk pelatihan
    eval_dataset=tokenized_datasets["validation"],  # Dataset untuk evaluasi
    data_collator=data_collator,  # Padding dinamis
    tokenizer=tokenizer,  # Tokenizer yang digunakan
)

<ipython-input-16-b271f254d359>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
# Melatih model
trainer.train()

Step,Training Loss
500,0.516100
1000,0.292300


TrainOutput(global_step=1377, training_loss=0.3368308926806038, metrics={'train_runtime': 229.1868, 'train_samples_per_second': 48.013, 'train_steps_per_second': 6.008, 'total_flos': 405114969714960.0, 'train_loss': 0.3368308926806038, 'epoch': 3.0})

In [18]:
# Membuat prediksi pada dataset validasi
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [19]:
# Menggunakan argmax untuk mengambil label dengan probabilitas tertinggi
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [20]:
# Memuat metrik evaluasi GLUE (MRPC)
import evaluate

metric = evaluate.load("glue", "mrpc")
# Menghitung metrik evaluasi
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8455882352941176, 'f1': 0.8911917098445595}

In [21]:
# Fungsi untuk menghitung metrik
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds  # Membagi logits dan label
    predictions = np.argmax(logits, axis=-1)  # Memilih prediksi dengan nilai tertinggi
    return metric.compute(predictions=predictions, references=labels)

In [22]:
# Mengatur ulang parameter pelatihan dengan evaluasi di setiap epoch
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

# Memuat ulang model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Membuat Trainer dengan metrik tambahan
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Metrik evaluasi
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-22-986ee46a9a41>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
# Melatih ulang model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.371126,0.857843,0.901024
2,0.527200,0.459970,0.845588,0.893401
3,0.279600,0.774146,0.848039,0.894915


TrainOutput(global_step=1377, training_loss=0.3341553964386319, metrics={'train_runtime': 215.6792, 'train_samples_per_second': 51.02, 'train_steps_per_second': 6.384, 'total_flos': 405114969714960.0, 'train_loss': 0.3341553964386319, 'epoch': 3.0})